# Apache Spark Tutorial

University of Liège                                                      
Joeri Hermans (joeri.hermans@doct.ulg.ac.be)                         

This notebook will guide the students of "Large-Scale Distributed Systems" through their initial steps with [Apache Spark](https://spark.apache.org).

## Downloading, unpacking and moving

In [14]:
# Download the latest version of Apache Spark.
!rm -r spark-2.2.0-bin-hadoop2.7.tgz
!wget -q http://apache.tt.co.kr/spark/spark-2.2.0/spark-2.2.0-bin-hadoop2.7.tgz

rm: cannot remove 'spark-2.2.0-bin-hadoop2.7': No such file or directory


In [20]:
# Unpack the framework.
!rm -r apache-spark
!tar -xzf spark-2.2.0-bin-hadoop2.7.tgz
!mv spark-2.2.0-bin-hadoop2.7 apache-spark
!ls -al

rm: cannot remove 'apache-spark': No such file or directory
tar (child): spark-2.2.0-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
total 20
drwxr-xr-x  4 joeri joeri 4096 Nov 15 01:42 .
drwxr-xr-x 10 joeri joeri 4096 Nov 15 00:36 ..
drwxr-xr-x 12 joeri joeri 4096 Jul  1 01:09 apache-spark
drwxr-xr-x  2 joeri joeri 4096 Nov 15 00:48 .ipynb_checkpoints
-rw-r--r--  1 joeri joeri 2623 Nov 15 01:41 spark-tutorial.ipynb


In [23]:
# Move it to the desired location. Make sure destination is writable.
!rm -rf /opt/apache-spark
!mv apache-spark /opt/

In [25]:
# Now, in order to use the libraries, make sure the libs are included in $PATH and $PYTHONPATH.
# Add this to your .bashrc or .zshrc depending on your shell.
!export SPARK_HOME=/opt/apache-spark
!export PYTHONPATH="$SPARK_HOME/python/:$SPARK_HOME/python/lib/py4j-0.10.4-src.zip:$PYTHONPATH"
!export PATH="$SPARK_HOME/bin:$PATH"

## Apache Spark

Test your installation by importing the Python module. If you did everything correctly, this should not produce any errors.

In [26]:
import pyspark

In the following sections we give a basic introduction to most of the concepts in Spark, and how (not) to use them. If you whish to implement something specific, you can always refer to the [documentation](https://spark.apache.org/docs/latest/quick-start.html).